In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from train_set import Training_set
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import  classification_report
from numpy import mean, std
from sklearn.svm import SVC

In [36]:
bio_features = pd.read_csv("bio_data.csv", index_col= 0)
#chi_features = pd.read_csv('../../../Chi_data.csv',index_col=0)
#rfe_features = pd.read_csv('../../RFE_FEATURES.csv',index_col=0)
X = bio_features
#Y has the last col with labels
y = Training_set.iloc[:, -1]
# split the dataset to train and test, the 2nd is for the non feature selection
Training_data = pd.merge(X, y, left_index=True, right_index=True)
print(Training_data)

X = Training_data.iloc[:, :-1]
#Y has the last col with labels
y = Training_data.iloc[:, -1]
print(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

           69274433-70345552  196908262-196937230  19049822-20716660  \
Array.129                 -1                    0                  0   
Array.34                   0                    0                  0   
Array.67                   0                   -1                  0   
Array.24                  -1                    0                  0   
Array.22                   0                    0                  1   
...                      ...                  ...                ...   
Array.10                   0                    0                  0   
Array.123                  0                    1                  1   
Array.100                 -1                    0                  0   
Array.134                  0                    1                  1   
Array.130                 -1                    0                  0   

           131304839-132036965  70574871-71644041  97852156-98629015  \
Array.129                    1                  1              

# NESTED CV

In [37]:
# Split the generated data into training and testing sets
skf_outer = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
actual_scores = []

# Define the parameter grid for the random forest classifier
param_grid = {
    'C': [0.1,0.3,0.5,1],
    'kernel': ['linear', 'rbf'],
    'decision_function_shape': ['ovo'],
    'gamma': ['scale','auto',0.1,0.5,1,5]
}

for train_index_inner, test_index_inner in skf_outer.split(X_train, y_train):
    X_train_inner, X_test_inner = X_train.iloc[train_index_inner], X_train.iloc[test_index_inner]
    y_train_inner, y_test_inner = y_train.iloc[train_index_inner], y_train.iloc[test_index_inner]

    # parameter tuning for the random forest classifier using GridSearchCV
    clf = SVC()

    # Inner cross-validation loop for hyperparameter tuning
    skf_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    grid_search = GridSearchCV(clf, param_grid, scoring='accuracy', cv=skf_inner)

    grid_search.fit(X_train_inner, y_train_inner)
    best_model = grid_search.best_estimator_
    yhat = best_model.predict(X_test_inner)
    acc= accuracy_score(y_test_inner,yhat)
    actual_scores.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, grid_search.best_score_, grid_search.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(actual_scores), std(actual_scores)))

>acc=0.875, est=0.937, cfg={'C': 0.5, 'decision_function_shape': 'ovo', 'gamma': 'scale', 'kernel': 'linear'}
>acc=0.938, est=0.906, cfg={'C': 0.1, 'decision_function_shape': 'ovo', 'gamma': 'scale', 'kernel': 'linear'}
>acc=0.938, est=0.922, cfg={'C': 0.1, 'decision_function_shape': 'ovo', 'gamma': 'scale', 'kernel': 'linear'}
>acc=0.875, est=0.922, cfg={'C': 0.1, 'decision_function_shape': 'ovo', 'gamma': 'scale', 'kernel': 'linear'}
>acc=1.000, est=0.938, cfg={'C': 0.3, 'decision_function_shape': 'ovo', 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 0.925 (0.047)


In [41]:
# Split the generated data into training and testing sets
skf_outer2 = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
actual_scores2 = []


for train_index_inner, test_index_inner in skf_outer2.split(X_train, y_train):
    X_train_inner, X_test_inner = X_train.iloc[train_index_inner], X_train.iloc[test_index_inner]
    y_train_inner, y_test_inner = y_train.iloc[train_index_inner], y_train.iloc[test_index_inner]



    # Inner cross-validation loop for hyperparameter tuning

    SV = SVC(C=  0.1, decision_function_shape = 'ovo', gamma = 'scale', kernel = 'linear')

    clf2 = SV.fit(X_train_inner, y_train_inner)

    yhat2 = clf2.predict(X_test_inner)
    acc2= accuracy_score(y_test_inner,yhat2)
    actual_scores2.append(acc2)
    # report progress
    print('>acc=%.3f' % (acc2))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(actual_scores2), std(actual_scores2)))

>acc=0.875
>acc=0.938
>acc=0.938
>acc=0.875
>acc=1.000
Accuracy: 0.925 (0.047)


1o 0.912
2o 0.925
3o 0.925

In [44]:
SVM_final = SVC(C=  0.1, decision_function_shape = 'ovo', gamma = 'scale', kernel = 'linear')

SVM_final.fit(X_train, y_train)
# Evaluate the trained classifier on the test set
y_pred_ = SVM_final.predict(X_test)
score_total = accuracy_score(y_test, y_pred_)
precision = metrics.precision_score(y_test, y_pred_, average='macro')
recall = metrics.recall_score(y_test, y_pred_, average='macro')
f1_score = metrics.f1_score(y_test, y_pred_, average='macro')

print(" Accuracy: {:.2f}%".format(score_total * 100))
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1-score: {f1_score:.3f}')

 Accuracy: 90.00%
Precision: 0.926
Recall: 0.889
F1-score: 0.892


In [47]:
cr = classification_report(y_test, y_pred_)
print(cr)

              precision    recall  f1-score   support

       HER2+       1.00      1.00      1.00         7
         HR+       0.78      1.00      0.88         7
  Triple Neg       1.00      0.67      0.80         6

    accuracy                           0.90        20
   macro avg       0.93      0.89      0.89        20
weighted avg       0.92      0.90      0.90        20



In [30]:
import pickle
pickle.dump(SVM_final, open('SVM_bio.pkl', 'wb'))